In [1]:
# imageclassfunctions

import torch
import numpy as np
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
#import imageclassfunctions as icf

In [2]:
# Putting the above into functions, so they can be used later
def do_deep_learning(model, trainloader, epochs, print_every, criterion, optimizer):
    
    print('learning start')
    print_every = print_every
    steps = 0

    for e in range(epochs):
        running_loss = 0
        for ii, (inputs, labels) in enumerate(trainloader):
            steps += 1

            optimizer.zero_grad()

            # Forward and backward passes
            outputs = model.forward(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if steps % print_every == 0:
                print("Epoch: {}/{}... ".format(e+1, epochs),
                      "Loss: {:.4f}".format(running_loss/print_every))

                running_loss = 0
                
    print('learning done') 

In [3]:
def check_accuracy_on_valid(model, validloader, criterion):    
    correct = 0
    total = 0

    iii = 0

    with torch.no_grad():
        running_loss = 0
        for data in validloader:
            images, labels = data
            # added one line below
            #images, labels = images.to('cuda'), labels.to('cuda')
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            loss = criterion(outputs, labels)
            running_loss += loss.item()
            iii += 1

    print('Accuracy of the network on the valid images: %d %%' % (100 * correct / total))
    
    print("Loss: {:.4f}".format(running_loss/iii))

In [4]:
# train.py

data_dir = '20flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=32)

model = models.vgg19(pretrained=True)
model_name = "vgg19"

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
h_units = 512
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(25088, h_units)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(h_units, 20)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
model.classifier = classifier

lr = 0.001
 
# TODO: Train a model with a pre-trained network
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr)

epochnum = 3
do_deep_learning(model, trainloader, epochnum, 10, criterion, optimizer)   

check_accuracy_on_valid(model, validloader, criterion)

# TODO: Save the checkpoint 
checkpoint = {'model name': model_name,
              'Hidden units': h_units,
              'state_dict': model.state_dict()}

torch.save(checkpoint, 'checkpoint_3.pth')


learning start
Epoch: 1/3...  Loss: 2.2040
Epoch: 2/3...  Loss: 0.3070
Epoch: 2/3...  Loss: 0.4741
Epoch: 3/3...  Loss: 0.3483
learning done
Accuracy of the network on the valid images: 87 %
Loss: 0.4234
